In [ ]:
# Transformer 요약 모델 & 파인 튜닝
# - Hugging Face 라이브러리 적용
# - AI HUB 요약문 및 레포트 뉴스(news) 데이터셋 적용
# - 입력된 문장을 요약 모델을 통한 문장 요약
# 1. 학습 목표
# - 구조 최적화 및 파이프라인 단순화
# - AI HUB 요약문 및 레포트 뉴스(news) 데이터셋 전처리
# - 병렬 문장쌍 데이터셋 변환 전처리
# - 토크나이징 및 토크나이징 전처리
# - 베이스 모델 로드
# - LoRA(Low-Rank Adaptation) 설정, 특정 레이어에 작은 저차원 행렬(랭크 r)을 삽입해서 학습
# - LoRA(Low-Rank Adaptation) 모델, 메모리 효율성/빠른 학습/도메인 적용, base 모델에 여러 LoRA 모듈을 붙였다 떼었다 할 수 있음
# - 학습 args 설정
# - Trainer 정의
# - Trainer 실행
# - LoRA 적용된 모델 저장, LoRA모델/토크나이저
# - LoRA 적용된 모델 불러오기, 베이스모델/LoRA모델/토크나이저

In [ ]:
import torch
import numpy as np
import glob, json, re, os, random, csv

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(torch.__version__, device)

print("CUDA 사용 가능 여부:", torch.cuda.is_available())
print("PyTorch CUDA 버전:", torch.version.cuda)
print("빌드 정보:", torch.__version__)
if torch.cuda.is_available():
    print("사용 중인 GPU:", torch.cuda.get_device_name(0))

In [ ]:
# 데이터셋 전처리 - AI HUB 요약문 및 레포트 뉴스(news) 데이터셋 적용
import os
import json
import pandas as pd

# 기본 경로
base_dir = './llm_data/ai_hub_summary_news_r'

def parse_json_folder(input_dir, summary_type):
    rows = []
    for file_name in os.listdir(input_dir):
        if not file_name.endswith('.json'):
            continue
        file_path = os.path.join(input_dir, file_name)
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        passage = data['Meta(Refine)']['passage']
        doc_type = data['Meta(Acqusition)']['doc_type']

        if summary_type == 'short':
            for key in ['summary1', 'summary2']:
                summary = data['Annotation'].get(key)
                if summary:
                    rows.append({
                        'src': passage,
                        'tgt': summary,
                        'summary_type': 'short',
                        'domain': doc_type
                    })
        elif summary_type == 'long':
            summary3 = data['Annotation'].get('summary3')
            if summary3:
                rows.append({
                    'src': passage,
                    'tgt': summary3,
                    'summary_type': 'long',
                    'domain': doc_type
                })
    # 판다스 데이터프레이으로 생성 후 리턴
    return pd.DataFrame(rows)

# 학습 데이터 로딩
train_short = parse_json_folder(os.path.join(base_dir, 'train/2-3sent'), 'short')
train_long = parse_json_folder(os.path.join(base_dir, 'train/20per'), 'long')

# 검증 데이터 로딩
valid_short = parse_json_folder(os.path.join(base_dir, 'valid/2-3sent'), 'short')
valid_long = parse_json_folder(os.path.join(base_dir, 'valid/20per'), 'long')

# 확인
print('train short:', train_short)
print('train long:', train_long)
print('valid short:', valid_short)
print('valid long:', valid_long)

In [ ]:
# 데이터셋 전처리 - AI HUB 요약문 및 레포트 뉴스(news) 데이터셋 적용
import os
import json
import pandas as pd
import re

# 기본 경로
base_dir = './llm_data/ai_hub_summary_news_r'

# 텍스트 클리닝 함수
def clean_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    # URL 제거 (대소문자 구분 없이, http/https 단독 포함)
    text = re.sub(r"https?://[^\s]+", "", text, flags=re.IGNORECASE)  # 전체 URL 제거
    text = re.sub(r"\bhttps?\b", "", text, flags=re.IGNORECASE)       # http/https 단독 제거
    text = re.sub(r"\bHTTP\b", "", text, flags=re.IGNORECASE)         # HTTP 단독 제거
    text = re.sub(r"\bHTTPS\b", "", text, flags=re.IGNORECASE)        # HTTPS 단독 제거

    # 날짜 제거 (YYYY-MM-DD, YYYY년 MM월 DD일, YYYY.MM.DD)
    text = re.sub(r"\d{4}년 \d{1,2}월 \d{1,2}일.*", "", text)
    text = re.sub(r"\d{4}-\d{2}-\d{2}.*", "", text)
    text = re.sub(r"\d{4}\.\d{1,2}\.\d{1,2}", "", text)

    # 기자명 제거
    text = re.sub(r"\S*기자", "", text)

    # 기사 특수기호 제거
    text = re.sub(r"[▲▶◆■□◇※△○]", "", text)

    # 불필요한 공백 제거
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def parse_json_folder(input_dir, summary_type):
    rows = []
    for file_name in os.listdir(input_dir):
        if not file_name.endswith('.json'):
            continue
        file_path = os.path.join(input_dir, file_name)
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        passage = clean_text(data['Meta(Refine)']['passage'])
        doc_type = data['Meta(Acqusition)']['doc_type']

        if summary_type == 'short':
            for key in ['summary1', 'summary2']:
                summary = data['Annotation'].get(key)
                if summary:
                    rows.append({
                        'src': passage,
                        'tgt': clean_text(summary),
                        'summary_type': 'short',
                        'domain': doc_type
                    })
        elif summary_type == 'long':
            summary3 = data['Annotation'].get('summary3')
            if summary3:
                rows.append({
                    'src': passage,
                    'tgt': clean_text(summary3),
                    'summary_type': 'long',
                    'domain': doc_type
                })
    return pd.DataFrame(rows)

# 학습 데이터 로딩
train_short = parse_json_folder(os.path.join(base_dir, 'train/2-3sent'), 'short')
train_long = parse_json_folder(os.path.join(base_dir, 'train/20per'), 'long')

# 검증 데이터 로딩
valid_short = parse_json_folder(os.path.join(base_dir, 'valid/2-3sent'), 'short')
valid_long = parse_json_folder(os.path.join(base_dir, 'valid/20per'), 'long')

# 확인
print('train short 샘플:', train_short.head())
print('train long 샘플:', train_long.head())
print('valid short 샘플:', valid_short.head())
print('valid long 샘플:', valid_long.head())

In [ ]:
# 학습 데이터 저장
train_short.to_csv(os.path.join(base_dir, 'train/2-3sent/train.csv'), index=False, encoding='utf-8-sig')
train_long.to_csv(os.path.join(base_dir, 'train/20per/train.csv'), index=False, encoding='utf-8-sig')

# 검증 데이터 저장
valid_short.to_csv(os.path.join(base_dir, 'valid/2-3sent/valid.csv'), index=False, encoding='utf-8-sig')
valid_long.to_csv(os.path.join(base_dir, 'valid/20per/valid.csv'), index=False, encoding='utf-8-sig')

print("CSV 저장 완료: train/valid 데이터셋 분리 저장")

In [1]:
# 모델 & 토크나이저 로드
from transformers import MT5ForConditionalGeneration, T5Tokenizer
from peft import LoraConfig, get_peft_model, TaskType

# 베이스 모델명
model_name = "google/mt5-base"
# 토크나이저
tokenizer = T5Tokenizer.from_pretrained(model_name)
# 모델
model = MT5ForConditionalGeneration.from_pretrained(model_name)

# LoRA 설정
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,   # mT5는 Seq2Seq 구조
    r=16,                              # 랭크 (작을수록 가볍고 빠름)
    lora_alpha=32,                     # LoRA scaling factor
    lora_dropout=0.1,                  # 드롭아웃
    target_modules=["q", "v"]          # 주로 attention의 query, value projection에 적용
)

# 기존 베이스 mT5 모델에 LoRA 적용
model = get_peft_model(model, lora_config)

# 적용 확인
model.print_trainable_parameters()

# trainable params: 1,769,472 || all params: 584,170,752 || trainable%: 0.3029
# 전체 파라미터 수: 약 5억 8천만 개 (mT5-base 전체 크기)
# 학습 가능한 파라미터 수: 약 176만 개
# 학습 비율: 약 0.3%만 학습 → 나머지는 고정(frozen)
# 즉, LoRA 덕분에 전체 모델을 학습시키지 않고도 극히 일부 모듈만 학습해서 GPU 메모리와 시간 절약이 가능

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


trainable params: 1,769,472 || all params: 584,170,752 || trainable%: 0.3029


In [3]:
# CSV 데이터셋 로드
from datasets import load_dataset, concatenate_datasets
import os

# 기본 경로
base_dir = './llm_data/ai_hub_summary_news_r'
# 전체 데이터셋 로드(short/long 모두 포함)
dataset = load_dataset(
    'csv',
    data_files={
        'train_short': os.path.join(base_dir, 'train/2-3sent/train.csv'),
        'train_long': os.path.join(base_dir, 'train/20per/train.csv'),
        'valid_short': os.path.join(base_dir, 'valid/2-3sent/valid.csv'),
        'valid_long': os.path.join(base_dir, 'valid/20per/valid.csv')
    }
)

# 두 split을 합쳐서 하나의 데이터셋으로 만들기
train_concat = concatenate_datasets([
    dataset['train_short'].select(range(200)),
    dataset['train_long'].select(range(200))
])

valid_concat = concatenate_datasets([
    dataset['valid_short'].select(range(100)),
    dataset['valid_long'].select(range(100))
])

print('train_concat : ', len(train_concat))
print('valid_concat : ', len(valid_concat))

train_concat :  400
valid_concat :  200


In [4]:
# src, tgt 컬럼에 None이나 NaN이 있는지 확인
print("src 결측치 개수:", sum(x is None or str(x).lower() == "nan" for x in train_concat["src"]))
print("tgt 결측치 개수:", sum(x is None or str(x).lower() == "nan" for x in train_concat["tgt"]))
print("summary_type 결측치 개수:", sum(x is None or str(x).lower() == "nan" for x in train_concat["summary_type"]))

# 결측치 제거
train_concat = train_concat.filter(lambda x: x["tgt"] is not None and str(x["tgt"]).strip() != "")
valid_concat = valid_concat.filter(lambda x: x["tgt"] is not None and str(x["tgt"]).strip() != "")

print("src 결측치 개수:", sum(x is None or str(x).lower() == "nan" for x in train_concat["src"]))
print("tgt 결측치 개수:", sum(x is None or str(x).lower() == "nan" for x in train_concat["tgt"]))
print("summary_type 결측치 개수:", sum(x is None or str(x).lower() == "nan" for x in train_concat["summary_type"]))

src 결측치 개수: 0
tgt 결측치 개수: 1
summary_type 결측치 개수: 0
src 결측치 개수: 0
tgt 결측치 개수: 0
summary_type 결측치 개수: 0


In [5]:
# 데이터셋 전처리 함수 & 데이터셋 토큰화

# 데이터셋 전처리
def preprocess(batch):
    # summary_type을 prefix로 활용
    inputs = [ f'[{stype.upper()}] {src}' for src, stype in zip(batch['src'], batch['summary_type'])]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    labels = tokenizer(batch['tgt'], max_length=128, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# 합쳐진 데이터셋에 바로 토큰화 적용
tokenized_train = train_concat.map(preprocess, batched=True)
tokenized_valid = valid_concat.map(preprocess, batched=True)

In [6]:
# Trainer 파이프라인
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import DataCollatorForSeq2Seq

# DataCollator 적용
# Seq2SeqTrainer에서는 DataCollatorForSeq2Seq를 사용하면 자동으로 길이에 대해서 패딩을 맞춰준다
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# 학습 설정
training_args = Seq2SeqTrainingArguments(
    output_dir="./llm_models/results_lora_ai_hub_news_r",       # 결과 저장 경로
    eval_strategy="epoch",                                      # 매 epoch마다 평가
    learning_rate=5e-4,                                         # 학습률, # LoRA는 보통 조금 더 큰 lr 사용 가능
    per_device_train_batch_size=4,                              # 학습 배치 크기
    per_device_eval_batch_size=4,                               # 검증 배치 크기
    num_train_epochs=3,                                         # 학습 epoch 수
    weight_decay=0.01,                                          # 가중치 감쇠
    save_total_limit=2,                                         # 체크포인트 최대 개수
    logging_dir="./llm_models/results_lora_logs_ai_hub_news_r", # 로그 저장 경로
    logging_steps=50,                                           # 로그 출력 주기
    predict_with_generate=True                                  # 평가 시 generate() 사용
)

# Trainer 정의
trainer = Seq2SeqTrainer(
    model=model,    # LoRA 적용 모델
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    processing_class=tokenizer,
    data_collator=data_collator     # HuggingFace Trainer는 배치 단위로 묶을 때 모든 시퀀스 길이가 동일하게 맞추기 위함
)

In [7]:
# 학습 진행
trainer.train()

Epoch,Training Loss,Validation Loss
1,4.956400,2.207439
2,2.830300,1.937208
3,2.662600,1.744926


TrainOutput(global_step=300, training_loss=5.5949776713053385, metrics={'train_runtime': 2542.0755, 'train_samples_per_second': 0.471, 'train_steps_per_second': 0.118, 'total_flos': 1441764893786112.0, 'train_loss': 5.5949776713053385, 'epoch': 3.0})

In [ ]:
# 추론
from transformers import pipeline

# 학습된 모델과 토크나이저 로드
summarizer = pipeline( # Hugging Face pipeline은 모델과 토크나이저를 묶어서 간단히 추론 API
    "text2text-generation",
    model=model,
    tokenizer=tokenizer
)

# 테스트 입력 문장
test_text = """
한국 방위산업은 최근 몇 년간 세계 시장에서 빠르게 성장하고 있다. 
특히 K-방산으로 불리는 한국형 무기체계는 중동, 동남아시아, 유럽 등 다양한 지역에서 수출 계약을 성사시키며 주목받고 있다. 
대표적으로 K2 전차, K9 자주포, FA-50 경공격기 등이 해외 시장에서 경쟁력을 입증했다. 
정부는 방산 수출을 국가 전략 산업으로 육성하기 위해 적극적인 외교와 지원 정책을 펼치고 있으며, 
국내 기업들도 기술 고도화와 현지 맞춤형 생산을 통해 시장을 확대하고 있다. 
이러한 흐름은 단순한 무기 판매를 넘어, 국방 협력과 외교적 영향력 강화로 이어지고 있다는 평가가 나온다.
"""
# short 요약
# - 한국 방산은 K2 전차, K9 자주포, FA-50 등으로 세계 시장에서 수출 성과를 내며 성장하고 있다.

#Short 요약: <extra_id_0> 한국 방위산업은 최근 몇 년간 세계 시장에서 빠르게 성장하고 있다. 
# 특히 K-방산으로 불리는 한국형 무기체계는 중동, 동남아시아, 유럽 등 다양한 지역에서 수출 계약 성사시키며 주목

#Cleaned Short 요약: 한국 방위산업은 최근 몇 년간 세계 시장에서 빠르게 성장하고 있다.
#  특히 K-방산으로 불리는 한국형 무기체계는 중동, 동남아시아, 유럽 등 다양한 지역에서 수출 계약 성사시키며 주목

# 요약 생성
# Short 요약 생성
summary_short = summarizer(
    f"[SHORT] {test_text}",
    max_new_tokens=64,
    do_sample=True,             # 샘플링 활성화
    top_k=50,                   # 다양성 확보
    top_p=0.95,                 # nucleus sampling
    no_repeat_ngram_size=3,     # 반복 방지
    repetition_penalty=2.0,     # 반복 패널티
    early_stopping=True
)

# 후처리 함수 (long/short 공통 사용 가능)
def clean_summary(text):
    text = text.replace("<extra_id_0>", "").replace("[SHORT]", "").replace("[LONG]", "").strip()
    if text.startswith(","):
        text = text[1:].strip()
    sentences = text.split(". ")
    seen = set()
    cleaned = []
    for s in sentences:
        s = s.strip()
        # 오타 및 중복 표현 교정
        s = s.replace("방산로", "방산").replace("침산", "방산")
        s = s.replace("방유산업", "방위산업")
        s = s.replace("확대를 확대", "확대를 추진")
        if s and s not in seen:
            cleaned.append(s)
            seen.add(s)
    return ". ".join(cleaned)

result_short = summary_short[0]["generated_text"]
result_short = clean_summary(result_short)
print("Cleaned Short 요약:", result_short)

Device set to use cuda:0


Cleaned Short 요약: 한국 방위산업은 최근 몇 년간 세계 시장에서 빠르게 성장하고 있다. 특히 K-방산으로 불리는 한국형 무기체계는 중동, 동남아시아, 유럽 등 다양한 지역에서 수출 계약 성사시키며 주목


In [ ]:
# long 요약
# - 한국 방위산업은 K-방산 무기체계의 수출 확대와 정부 지원 정책을 바탕으로 세계 시장에서 경쟁력을 강화하고 있으며, 
# - 이는 단순한 무기 판매를 넘어 국방 협력과 외교적 영향력 확대에도 기여하고 있다.

#Cleaned Long 요약: FA-50 경공격기 등이 해외 시장에서 경쟁력을 입증했다. 
# 특히 K-방산으로 불리는 한국형 무기체계는 중동, 동남아시아, 유럽 등 다양한 지역에서 수출 계약 성사시키며 주목받고 있다. 
# 특수적으로 K2 전차, K9 자주포 등이 세계 시장에서 활약하고 있다. 
# 이러한 흐름은 단순한 무기 판매를 넘어, 국방 협력과 외교적 영향을 확대하기 위해 적극적인 외교와 지원 정책을 펼치고 있으며, 
# 국내 기업들도 기술 고도화와 현지 맞춤형 생산을 통해 시장 확대를 추진하고 있다

summary_long = summarizer(
    f"[LONG] {test_text}",
    max_new_tokens=256,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    no_repeat_ngram_size=3,
    repetition_penalty=2.0,
    early_stopping=True
)


# 반복문 제거
def clean_summary(text):
    text = text.replace("<extra_id_0>", "").replace("[LONG]", "").strip()
    # 불필요한 쉼표 제거
    if text.startswith(","):
        text = text[1:].strip()
    # 문장 단위 분리
    sentences = text.split(". ")
    seen = set()
    cleaned = []
    for s in sentences:
        s = s.strip()
        # 오타 및 중복 표현 교정
        s = s.replace("방산로", "방산").replace("침산", "방산")
        s = s.replace("방유산업", "방위산업")
        s = s.replace("확대를 확대", "확대를 추진")
        if s and s not in seen:
            cleaned.append(s)
            seen.add(s)
    return ". ".join(cleaned)

result = summary_long[0]["generated_text"]
result = clean_summary(result)
# result = result.replace("<extra_id_0>", "").replace("[LONG]", "").strip()
print("Cleaned Long 요약:", result)


Cleaned Long 요약: FA-50 경공격기 등이 해외 시장에서 경쟁력을 입증했다. 특히 K-방산으로 불리는 한국형 무기체계는 중동, 동남아시아, 유럽 등 다양한 지역에서 수출 계약 성사시키며 주목받고 있다. 특수적으로 K2 전차, K9 자주포 등이 세계 시장에서 활약하고 있다. 이러한 흐름은 단순한 무기 판매를 넘어, 국방 협력과 외교적 영향을 확대하기 위해 적극적인 외교와 지원 정책을 펼치고 있으며, 국내 기업들도 기술 고도화와 현지 맞춤형 생산을 통해 시장 확대를 추진하고 있다


In [ ]:
# LoRA 적용된 모델 저장
model.save_pretrained("./llm_models/summary_model_ai_hub_news_r_lora")
tokenizer.save_pretrained("./llm_models/summary_model_ai_hub_news_r_lora")

In [ ]:
from transformers import MT5ForConditionalGeneration, T5Tokenizer

# 모델 로드
model = MT5ForConditionalGeneration.from_pretrained(
    "./llm_models/summary_model_ai_hub_news_r_lora"
)

# 토크나이저 로드
tokenizer = T5Tokenizer.from_pretrained(
    "./llm_models/summary_model_ai_hub_news_r_lora"
)

In [ ]:
# 모델 로드 & 추론
from transformers import pipeline

# 학습된 모델과 토크나이저 로드
summarizer = pipeline( # Hugging Face pipeline은 모델과 토크나이저를 묶어서 간단히 추론 API
    "text2text-generation",
    model=model,
    tokenizer=tokenizer
)

# 테스트 입력 문장
test_text = """
지난주 열린 국제 AI 컨퍼런스에서는 생성형 AI의 윤리적 활용과
기업 내 도입 전략에 대한 다양한 논의가 이루어졌다.
특히 데이터 프라이버시와 저작권 문제에 대한 해결책이 주요 의제로 다뤄졌다.
"""

# 요약 생성
# summary = summarizer( # Greedy Decoding (탐욕적 디코딩)
#     f"[SHORT] {test_text}", # summary_type prefix 활용, prefix([SHORT], [LONG])
#     max_length=128,
#     min_length=20,
#     do_sample=False         # 샘플링 없이 greedy decoding으로 결정적 결과
# )
# Beam Search (빔 탐색)
summary = summarizer(
    f"[SHORT] {test_text}",
    max_length=128,
    num_beams=5,      # 빔 개수
    early_stopping=True
)
# Sampling?(샘플링)
# summary = summarizer(
#     f"[SHORT] {test_text}",
#     max_length=128,
#     do_sample=True,   # 샘플링 켬
#     top_k=50,         # 확률 상위 50개 중 선택
#     top_p=0.95,       # 누적 확률 95% 내에서 선택
#     num_return_sequences=3
# )

# summary[0] 리스트의 첫 번째 결과 (딕셔너리), summary[0]["generated_text"] 딕셔너리 안에서 모델이 실제로 생성한 요약문 텍스트
print('요약 결과:', summary[0]["generated_text"])
# for i, s in enumerate(summary_2):
#     print(f"요약 {i+1}: {s['generated_text']}")